# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 13 2023 2:15PM,255027,19,9027503,"Innogenix, LLC",Released
1,Jan 13 2023 2:15PM,255027,19,9027504,"Innogenix, LLC",Released
2,Jan 13 2023 2:15PM,255027,19,9027505,"Innogenix, LLC",Released
3,Jan 13 2023 2:15PM,255027,19,9027506,"Innogenix, LLC",Released
4,Jan 13 2023 2:15PM,255027,19,9027507,"Innogenix, LLC",Released
5,Jan 13 2023 2:15PM,255027,19,9027508,"Innogenix, LLC",Released
6,Jan 13 2023 2:15PM,255027,19,9027509,"Innogenix, LLC",Released
7,Jan 13 2023 2:15PM,255027,19,9027510,"Innogenix, LLC",Released
8,Jan 13 2023 2:15PM,255027,19,9027511,"Innogenix, LLC",Released
9,Jan 13 2023 2:15PM,255027,19,9027512,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,255023,Released,1
16,255024,Released,20
17,255025,Released,1
18,255026,Released,21
19,255027,Released,35


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255023,NaN,NaN,1.0
255024,NaN,NaN,20.0
255025,NaN,NaN,1.0
255026,NaN,NaN,21.0
255027,NaN,NaN,35.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254949,0.0,0.0,4.0
254976,8.0,26.0,14.0
254980,7.0,2.0,3.0
254992,0.0,0.0,8.0
254995,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254949,0,0,4
254976,8,26,14
254980,7,2,3
254992,0,0,8
254995,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254949,0,0,4
1,254976,8,26,14
2,254980,7,2,3
3,254992,0,0,8
4,254995,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254949,,,4
1,254976,8,26,14
2,254980,7,2,3
3,254992,,,8
4,254995,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC"
35,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc."
56,Jan 13 2023 1:45PM,255025,19,"GUSA Granules USA, Inc."
57,Jan 13 2023 1:39PM,255024,10,ISDIN Corporation
77,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC"
78,Jan 13 2023 1:13PM,255021,19,"GUSA Granules USA, Inc."
79,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC
80,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation
93,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC"
98,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",,,35
1,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",,,21
2,Jan 13 2023 1:45PM,255025,19,"GUSA Granules USA, Inc.",,,1
3,Jan 13 2023 1:39PM,255024,10,ISDIN Corporation,,,20
4,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC",,,1
5,Jan 13 2023 1:13PM,255021,19,"GUSA Granules USA, Inc.",,,1
6,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,,,1
7,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation,,1,12
8,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",,2,3
9,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",35,,
1,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",21,,
2,Jan 13 2023 1:45PM,255025,19,"GUSA Granules USA, Inc.",1,,
3,Jan 13 2023 1:39PM,255024,10,ISDIN Corporation,20,,
4,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC",1,,
5,Jan 13 2023 1:13PM,255021,19,"GUSA Granules USA, Inc.",1,,
6,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,1,,
7,Jan 13 2023 12:18PM,255011,10,ISDIN Corporation,12,1,
8,Jan 13 2023 10:01AM,254996,15,"Virtus Pharmaceuticals, LLC",3,2,
9,Jan 13 2023 10:01AM,254995,15,"Mizner Bioscience, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",35,,
1,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",21,,
2,Jan 13 2023 1:45PM,255025,19,"GUSA Granules USA, Inc.",1,,
3,Jan 13 2023 1:39PM,255024,10,ISDIN Corporation,20,,
4,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",35,NaN,NaN
1,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",21,NaN,NaN
2,Jan 13 2023 1:45PM,255025,19,"GUSA Granules USA, Inc.",1,NaN,NaN
3,Jan 13 2023 1:39PM,255024,10,ISDIN Corporation,20,NaN,NaN
4,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",35,0.0,0.0
1,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",21,0.0,0.0
2,Jan 13 2023 1:45PM,255025,19,"GUSA Granules USA, Inc.",1,0.0,0.0
3,Jan 13 2023 1:39PM,255024,10,ISDIN Corporation,20,0.0,0.0
4,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1274987,40,3.0,7.0
15,764983,12,2.0,0.0
18,255015,1,0.0,0.0
19,1020049,51,26.0,8.0
20,255026,21,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1274987,40,3.0,7.0
1,15,764983,12,2.0,0.0
2,18,255015,1,0.0,0.0
3,19,1020049,51,26.0,8.0
4,20,255026,21,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,40,3.0,7.0
1,15,12,2.0,0.0
2,18,1,0.0,0.0
3,19,51,26.0,8.0
4,20,21,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,15,Released,12.0
2,18,Released,1.0
3,19,Released,51.0
4,20,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20
Status,,,,,
Completed,7.0,0.0,0.0,8.0,0.0
Executing,3.0,2.0,0.0,26.0,0.0
Released,40.0,12.0,1.0,51.0,21.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20
0,Completed,7.0,0.0,0.0,8.0,0.0
1,Executing,3.0,2.0,0.0,26.0,0.0
2,Released,40.0,12.0,1.0,51.0,21.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20
0,Completed,7.0,0.0,0.0,8.0,0.0
1,Executing,3.0,2.0,0.0,26.0,0.0
2,Released,40.0,12.0,1.0,51.0,21.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()